# Feed forward network

In this exercise you shall develop our own building blocks for constructing a feed forward network.
We will follow a modular approach so that we can use these building blocks in feed forward architecture of our choice.

We will follow the logic of computation graphs where the layers and the loss have the characteristics of the compute nodes in terms of locality and ability to communicate with upstream and downstream blocks.

Instead of defining the forward and backward steps as functions that need to pass around cached variables, we will implement the compute nodes as statefull objects - instantiations of python classes with forward and backward methods.

We will then conscruct a 2 layer neural network and use our newly developed functionality to predict the target values and compute the parameter gradients.

Work through the cells below and complete the tasks indicated by <span style="color:red">**TODO**</span> here below and in the script `ann_code/layers.py` (replace `pass` with the appropriate code).

In [1]:
# necessary initialization
%load_ext autoreload
%autoreload 2

import torch

In [2]:
# load data
from ann_code.helpers import load_data
in_data, labels = load_data(filename='./ann_data/toy_data.csv') # correct filename if necessary

# get data dimensions
num_inst, num_dim = in_data.shape
print(f"Number of instances: {num_inst}, input dimensions: {num_dim}.")

Number of instances: 90, input dimensions: 3.


## 1) Forward pass

We first work on the forward pass functionality of our layer objects.

### Linear layer

We start by defyining the linear layer.
Complete the `__init__` and `forward` methods of the `Linear` class in `ann_code/layers.py`.

The class object instances shall be initialized with the linear function parameters (weight and bias) as the instance attributes.
The other local information (inputs, outputs and their gradients) shall be also defined as the instance object attributes and will be populated by the `forward` and `backward` methods.

In [3]:
# after implementing Linear class, check it here
from ann_code.layers import Linear

# initiate w and b buffers
# we use these for initiating the model parameters instead of the usual random init
# this is to make sure that yours and mine results match
w_buffer = torch.logspace(start=0.1, end=10, steps=1000)
b_buffer = torch.logspace(start=0.1, end=10, steps=1000, base=2)

# linear layer dimensions
in_features = num_dim
out_features = 10
################################################################################
### START OF YOUR CODE                                                         #
### TODO: initiate a linear layer instance                                     #
################################################################################
# initialize linear layer parameters from the buffers
# first extract from the buffers the necessary number of elements 
# followed by view() to get the correct shape
# e.g. for 2x3 w matrix with 6 elements in total do 
# w = w_buffer[:6].view(2, 3)

w = w_buffer[:out_features*in_features].view(out_features, num_dim)
b = b_buffer[:out_features].view(1, out_features)
#pass

# instantiate the linear layer object
linear = Linear(w, b)
#pass

################################################################################
### END OF YOUR CODE                                                           #
################################################################################

# forward pass in_data through the layer
outputs = linear.forward(in_data)

# check outputs for the first two data instances
print(f'Your outputs {outputs[:2,:]}')

Your outputs tensor([[ 1.0220,  1.0258,  1.0295,  1.0329,  1.0361,  1.0391,  1.0418,  1.0441,
          1.0462,  1.0479],
        [-0.4527, -0.5533, -0.6615, -0.7779, -0.9030, -1.0374, -1.1819, -1.3370,
         -1.5037, -1.6827]])


Expected outputs

`tensor([[ 1.0220,  1.0258,  1.0295,  1.0329,  1.0361,  1.0391,  1.0418,  1.0441,
          1.0462,  1.0479],
        [-0.4527, -0.5533, -0.6615, -0.7779, -0.9030, -1.0374, -1.1819, -1.3370,
         -1.5037, -1.6827]])`

### ReLU nonlinearity

We next defined the class for the Rectified Linear Unit which is an element-wise operation defined as $ReLU(x) = max(0, x).$

Complete the `forward` methods of the `Relu` class in `ann_code/layers.py`. Note that in this case, there are no parameters that should be included in the object instances as initial states.

In [4]:
# After implementing Relu class, check it here
from ann_code.layers import Relu

# relu instance
relu = Relu()

# forward pass in_data through the layer
outputs = relu.forward(in_data)

# check outputs for the first two data instances
print(f'Your outputs {outputs[:2,:]}')

Your outputs tensor([[0.8872, 0.0000, 0.3707],
        [0.0000, 1.3094, 0.0000]])


Expected outputs

`tensor([[0.8872, 0.0000, 0.3707],
        [0.0000, 1.3094, 0.0000]])`

### Define network with on hidden layer

We use the linear and relu classes to create a network with the following architecture. 
We combine the layers through the `Model` class that I defined for you in the `ann_code/layers.py`

We will add the MSE less in a later step, now do just the forward pass through the layers to obtain the predicitons.

<center><img src="net_diagram.png">



In [32]:
# work with Model class to do the forward pass through the network
from ann_code.layers import Model

################################################################################
### START OF YOUR CODE                                                         #
### TODO: initiate all layers of nerual network                                #
################################################################################
# initialize parameters for all layers from the w_buffer and b_buffer
# first extract from the buffers the necessary number of elements 
# followed by view() to get the correct shape
# start after the last previously used element of the buffer for every new parameter from the same buffer
# e.g. for 2x3 matrix w1 with 6 elements and 3x3 matrix w2 with 9 elements do
# w1 = w_buffer[:6].view(2, 3)
# w2 = w_buffer[6:15].view(3, 3)

out_features_h1 = 4
in_features_h1 = num_dim
in_features_h2 = out_features_h1
out_features_h2 = 1

w1 = w_buffer[:out_features_h1*in_features_h1].view(out_features_h1, in_features_h1)
b1 = b_buffer[:out_features_h1].view(1, out_features_h1)
w2 = w_buffer[out_features_h1*in_features_h1 : out_features_h1*in_features_h1 + out_features_h2*in_features_h2].view(out_features_h2, in_features_h2)
b2 = b_buffer[out_features_h1 : out_features_h1 + out_features_h2].view(1, out_features_h2)

#pass

# instantiate the linear layer object
lin1 = Linear(w1, b1)
relu1 = Relu()
lin2 = Linear(w2, b2)
#pass

# define all necessary layers as instances of the Linear and Relu classes
#pass

# create a list with the layers to be passed to Model(layers)
layers = [lin1, relu1, lin2]
#pass

################################################################################
### END OF YOUR CODE                                                           #
################################################################################


# forward pass in_data through all layers to get predictions
model = Model(layers)
ypred = model.forward(in_data)


# check outputs for the first two data instances
print(f'Your outputs {ypred[:2,:]}')

Your outputs tensor([[8.1458],
        [1.1016]], grad_fn=<SliceBackward0>)


Expected output

`tensor([[8.1458],
        [1.1016]])`

## 3) MSE loss

We use the MSE loss functions defined in `ann_code/linear_regression.py` to get the mse loss for our predictions and the corresponding gradients.

In [33]:
# use mse functions defined for linear regression to get the MSE and gradient with respect to predictions
from ann_code.linear_regression import mse_forward, mse_backward

loss, mse_cache = mse_forward(ypred, labels)
ypredgrad, _ = mse_backward(mse_cache)

## 3) Backward propagation

Finally, you need to implement the `backward` methods in for the `Linear` and `Relu` classes.

Remember that you need to use the chain rule and combine the local and the upstream gradient to obtain the global gradients. Do not forget that ReLu is an element-wise operation.

In [34]:
# After implementing the backward passes of Linear class test it here

# do the backward pass of last linear layer
lin2.backward(torch.ones(num_inst, 1))

# check global gradients
print(f'Global gradient of loss with respect to weight parameters {lin2.w_g}')
print(f'Global gradient of loss with respect to bias parameters {lin2.b_g}')
print(f'Global gradient of loss with respect to linear layer inputs {lin2.ins_g[:2,:]}')

Global gradient of loss with respect to weight parameters tensor([[106.2968, 108.7577, 111.4530, 114.4143]], grad_fn=<MmBackward0>)
Global gradient of loss with respect to bias parameters tensor([[90.]])
Global gradient of loss with respect to linear layer inputs tensor([[1.6555, 1.6937, 1.7328, 1.7728],
        [1.6555, 1.6937, 1.7328, 1.7728]])


Expected results

`Global gradient of loss with respect to weight parameters tensor([[106.2968, 108.7577, 111.4530, 114.4143]])`

`Global gradient of loss with respect to bias parameters tensor([[90.]])`

`Global gradient of loss with respect to linear layer inputs tensor([[1.6555, 1.6937, 1.7328, 1.7728],
        [1.6555, 1.6937, 1.7328, 1.7728]])`

In [35]:
# After implementing the backward passes of relu class test it here

# do the backward pass of relu
relu1.backward(torch.arange(num_inst*4).view(num_inst, 4))

# check global gradients
print(f'Global gradient of loss with respect to relu inputs {relu1.ins_g[:2,:]}')

Global gradient of loss with respect to relu inputs tensor([[0., 1., 2., 3.],
        [0., 0., 0., 0.]])


Expected results

`Global gradient of loss with respect to relu inputs tensor([[0., 1., 2., 3.],
        [0., 0., 0., 0.]])`

## Complete backward pass

We shall use the Model class to get the gradients of all the layers and their parameters with respect to the loss.

In [36]:
from ann_code.helpers import grad_model

# do the backward pass through the model
model.backward(ypredgrad)

# print out your gradients of loss with respect to the parameters of the 1st model layer
print(f'Your dLoss/dW1: {model.layers[0].w_g}')
print(f'Your dLoss/db1: {model.layers[0].b_g}')
print(f'Your dLoss/dins: {model.layers[0].ins_g[:2, :]}')

# print out correct gradients of loss with respect to the parameters of the 1st model layer
# these should be the same as your gradients from above
model_check = grad_model(model, in_data, labels)
print(f'Correct dLoss/dW1: {model_check.layers[0].w_g}')
print(f'Correct dLoss/db1: {model_check.layers[0].b_g}')
print(f'Correct dLoss/dins: {model_check.layers[0].ins_g[:2, :]}')

Your dLoss/dW1: tensor([[10.4693,  6.8379,  4.1449],
        [10.5790,  7.0695,  4.3389],
        [10.8324,  7.2315,  4.4382],
        [11.0693,  7.3818,  4.5600]], grad_fn=<MmBackward0>)
Your dLoss/db1: tensor([[31.2568, 31.9208, 32.6484, 33.4148]], grad_fn=<SumBackward1>)
Your dLoss/dins: tensor([[1.6884, 1.7274, 1.7673],
        [0.0000, 0.0000, 0.0000]], grad_fn=<SliceBackward0>)
Correct dLoss/dW1: tensor([[10.4693,  6.8379,  4.1449],
        [10.5790,  7.0695,  4.3389],
        [10.8324,  7.2315,  4.4382],
        [11.0693,  7.3818,  4.5600]], grad_fn=<MmBackward0>)
Correct dLoss/db1: tensor([[31.2568, 31.9208, 32.6484, 33.4148]], grad_fn=<SumBackward1>)
Correct dLoss/dins: tensor([[1.6884, 1.7274, 1.7673],
        [0.0000, 0.0000, 0.0000]], grad_fn=<SliceBackward0>)


## 4) Multilayer feed forward network

Finally, use your `Linear` and `Relu` classes and combine them with the `Model` class to construct a more complicated network.

Define a network with the following architecture:
Linear: input_dim = 3, output_dim = 5 -> Relu ->
Linear: input_dim = 5, output_dim = 10 -> Relu ->
Linear: input_dim = 10, output_dim = 4 -> Relu ->
Linear: input_dim = 4, output_dim = 1

Initialize all the linear layers with parameters W and b sampled randomly from standardat normal distribution.

Combine the layers using the `Model` class and get the predictions (`forward` method).

Use the MSE forward and backward functions to get the loss and the gradient with respect to the predictions.

Use the `backward` method of `Model` to get all the gradients.

In [38]:
################################################################################
### START OF YOUR CODE                                                         #
### TODO: define mffn as instance of Model class                               #
################################################################################

# instantiate all layers

out_features_h1 = 5
in_features_h1 = num_dim
in_features_h2 = out_features_h1
out_features_h2 = 10
in_features_h3 = out_features_h2
out_features_h3 = 4
in_features_h4 = out_features_h3
out_features_h4 = 1

w1 = w_buffer[:out_features_h1*in_features_h1].view(out_features_h1, in_features_h1)
b1 = b_buffer[:out_features_h1].view(1, out_features_h1)
w2 = w_buffer[out_features_h1*in_features_h1 : out_features_h1*in_features_h1 + out_features_h2*in_features_h2].view(out_features_h2, in_features_h2)
b2 = b_buffer[out_features_h1 : out_features_h1 + out_features_h2].view(1, out_features_h2)
w3 = w_buffer[out_features_h1*in_features_h1 + out_features_h2*in_features_h2 : out_features_h1*in_features_h1 + out_features_h2*in_features_h2 + out_features_h3*in_features_h3].view(out_features_h3, in_features_h3)
b3 = b_buffer[out_features_h1 + out_features_h2 : out_features_h1 + out_features_h2 + out_features_h3].view(1, out_features_h3)
w4 = w_buffer[out_features_h1*in_features_h1 + out_features_h2*in_features_h2 + out_features_h3*in_features_h3 : out_features_h1*in_features_h1 + out_features_h2*in_features_h2 + out_features_h3*in_features_h3 + out_features_h4*in_features_h4].view(out_features_h4, in_features_h4)
b4 = b_buffer[out_features_h1 + out_features_h2 + out_features_h3 : out_features_h1 + out_features_h2 + out_features_h3 + out_features_h4].view(1, out_features_h4)   #todo: out_features_h4 or 1? !!!



#pass

# instantiate the linear layer object
lin1 = Linear(w1, b1)
relu1 = Relu()
lin2 = Linear(w2, b2)
relu2 = Relu()
lin3 = Linear(w3, b3)
relu3 = Relu()
lin4 = Linear(w4, b4)

layers = [lin1, relu1, lin2, relu2, lin3, relu3, lin4]

#pass

# define model using Model class
mffn = Model(layers)

# forward, mse, backward
#pass
ypred = mffn.forward(in_data)

loss, mse_cache = mse_forward(ypred, labels)    #todo this is prob gonna crash
ypredgrad, _ = mse_backward(mse_cache)

mffn.backward(ypredgrad)



################################################################################
### END OF YOUR CODE                                                           #
################################################################################

tensor([[2.6398e+08, 2.7008e+08, 2.7631e+08],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.6396e+08, 3.7236e+08, 3.8095e+08],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.0823e+08, 3.1535e+08, 3.2263e+08],
        [1.3905e+08, 1.4226e+08, 1.4554e+08],
        [3.1995e+07, 3.2734e+07, 3.3489e+07],
        [3.5650e+08, 3.6472e+08, 3.7314e+08],
        [3.0823e+08, 3.1534e+08, 3.2262e+08],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [5.7920e+08, 5.9257e+08, 6.0625e+08],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [4.8183e+08, 4.9295e+08, 5.0433e+08],
        [1.6493e+08, 1.6873e+08, 1.7263e+08],
        [7.5391e+07, 7.7131e+07, 7.8911e+07],
        [6.1651e+07, 6.3074e+07, 6.4530e+07],
        [4.6067e+07, 4.7131e+07, 4.8218e+07],
        [5.4288e+08, 5.5541e+08, 5.6823e+08],
        [3.8479e+08, 3.9367e+08, 4.0275e+08],
        [4.3061e+08, 4.4055e+08, 4.5071e+08],
        [1.2267e+08, 1.2550e+08, 1.2840e+08],
        [0.0000e+00, 0.0000e+00, 0

#### Check model architecture

In [39]:
# check architecture
from ann_code.helpers import check_architecture

check_architecture(mffn)

Your NN architecture definitions seems CORRECT.


#### Check gradient computation

In [43]:
# print out your gradients of loss with respect to the parameters of the 1st model layer
print(f'Your dLoss/dW1: {mffn.layers[0].w_g}')
print(f'Your dLoss/db1: {mffn.layers[0].b_g}')
print(f'Your dLoss/dins: {mffn.layers[0].ins_g[:2, :]}') 
    
# print out correct gradients of loss with respect to the parameters of the 1st model layer
# these should be the same as your gradients from above
model_check = grad_model(mffn, in_data, labels)
print(f'Correct dLoss/dW1: {model_check.layers[0].w_g}')
print(f'Correct dLoss/db1: {model_check.layers[0].b_g}')
print(f'Correct dLoss/dins: {model_check.layers[0].ins_g[:2, :]}')

Your dLoss/dW1: tensor([[1.3177e+09, 7.6940e+08, 5.1239e+08],
        [1.3458e+09, 7.9341e+08, 5.2419e+08],
        [1.3800e+09, 8.1131e+08, 5.3599e+08],
        [1.4155e+09, 8.3465e+08, 5.4294e+08],
        [1.4540e+09, 8.4408e+08, 5.6341e+08]], grad_fn=<MmBackward0>)
Your dLoss/db1: tensor([[3.6906e+09, 3.7709e+09, 3.8548e+09, 3.9402e+09, 4.0251e+09]],
       grad_fn=<SumBackward1>)
Your dLoss/dins: tensor([[2.6398e+08, 2.7008e+08, 2.7631e+08],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]], grad_fn=<SliceBackward0>)
Correct dLoss/dW1: tensor([[1.3177e+09, 7.6940e+08, 5.1239e+08],
        [1.3458e+09, 7.9341e+08, 5.2419e+08],
        [1.3800e+09, 8.1131e+08, 5.3599e+08],
        [1.4155e+09, 8.3465e+08, 5.4294e+08],
        [1.4540e+09, 8.4408e+08, 5.6341e+08]], grad_fn=<MmBackward0>)
Correct dLoss/db1: tensor([[3.6906e+09, 3.7709e+09, 3.8548e+09, 3.9402e+09, 4.0251e+09]],
       grad_fn=<SumBackward1>)
Correct dLoss/dins: tensor([[2.6398e+08, 2.7008e+08, 2.7631e+08],
        [0.0000e